In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps
from lava.proc import io
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import RefPort

from lava.lib.dl import netx

from torch.utils.data.dataloader import DataLoader
from Loss import KDLoss
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision

%load_ext autoreload

%autoreload 2

# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:1')
    #device = torch.device("cuda")     
else:
    device = torch.device("cpu")

In [3]:
net = netx.hdf5.Network(net_config='net_snn.net')
print(net)

OSError: Unable to open file (truncated file: eof = 3464, sblock->base_addr = 0, stored_eof = 3474)

In [10]:
print(f'There are {len(net)} layers in network:')

for l in net.layers:
    print(f'{l.block:5s} : {l.name:10s}, shape : {l.shape}')

There are 3 layers in network:
Dense : Process_13, shape : (64,)
Dense : Process_16, shape : (128,)
Dense : Process_19, shape : (2,)


In [5]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [0]

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S1'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23c
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

SNR = SNR_list[0]
i = 1
rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i,
        N_reservoir=16,
        spectral_radius=0.2,
        sparsity=0.4,
        noise=1e-6,
        lut_activation=False,  # True,
        tanh_lut=tanh_lut,
        input_scale=25,  #40, #50, # 25,
        reservoir_input_scale = 8,  #4,  #5,
        show_wout=False,
        output_folder= output_folder,
        debug=debug,
        use_fpga= None,
        sock= None,  # usock
        addr = None) # addr

train_input, train_label, test_input, test_label = rc.run()
train_mean = np.mean(train_input)
train_std = np.std(train_input)

train_input = (train_input - train_mean) / train_std
test_input = (test_input - train_mean) / train_std

In [7]:
nb_inputs  = 2
nb_hidden  = 96
nb_outputs = 2

time_step = 1e-3
nb_steps  = 100

batch_size = 32

import scipy.io

from dataset import Dataset, SpikingDataset, ARegDataset, RegSpikingDataset, RegTorchSpikingDataset, RegTorchFixedSpikingDataset, TestDataset
train_data = np.load('input_snn.npy', allow_pickle=True)
train_label = np.load('label_snn.npy', allow_pickle=True)
#train_data = RegTorchFixedSpikingDataset(train_input, train_label, nb_inputs, nb_steps)
print(train_data.shape)
train_data = TestDataset(train_data, train_label)
if np.isscalar(train_data[0][1]):
    print("True")
print(len(train_data))
print(train_data[0][1].shape)
print(train_data[0][0].shape)

(59,)
59
torch.Size([128, 2])
torch.Size([128, 32, 100])


In [13]:
for i in range(10):
    x, y = train_data[i]
    print(x.shape, y.shape)
    print(data[:, 10])

torch.Size([128, 32, 100]) torch.Size([128, 2])


TypeError: tuple indices must be integers or slices, not tuple

In [9]:
from lava.proc import io

# Instantiate the processes
dataloader = io.dataloader.SpikeDataloader(dataset=train_data, interval=100)
output_logger = io.sink.RingBuffer(shape=net.out_layer.shape, buffer=nb_steps)
#gt_logger = io.sink.RingBuffer(shape=(2,), buffer=nb_steps)
# reset
for i, l in enumerate(net.layers[:-1]):
    u_resetter = io.reset.Reset(interval=nb_steps, offset=i)
    v_resetter = io.reset.Reset(interval=nb_steps, offset=i)
    u_resetter.connect_var(l.neuron.u)
    v_resetter.connect_var(l.neuron.v)

# Connect the input to the network:
#dataloader.ground_truth.connect(gt_logger.a_in)
dataloader.s_out.connect(net.inp)

# Connect network-output to the output process
net.out.connect(output_logger.a_in)

NameError: name 'net' is not defined

In [15]:
pretrained_model = torch.load('./net_dnn.pth', map_location=torch.device('cpu'))
class DNNNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()

        self.fc1 = nn.Linear(input_size, output_size)
        #self.fc2 = nn.Linear(64, 128)
        #self.fc3 = nn.Linear(128, 128)
        #self.fc4 = nn.Linear(128, 2)
        self.act = nn.ReLU()

    def forward(self,x):
        x = x.flatten(start_dim=1)
        x = self.fc1(x)
        #x = self.fc1(x)
        #x = self.act(x)
        #x = self.fc2(x)
        #x = self.act(x)
        #x = self.fc3(x)
        #x = self.act(x)
        #x = self.fc4(x)
        return x
    
net_dnn = DNNNetwork(2 * 100, 2).to(device)
net_dnn.load_state_dict(pretrained_model)

<All keys matched successfully>

In [16]:
class CustomRunConfig(Loihi1SimCfg):
    def select(self, proc, proc_models):
        # customize run config to always use float model for io.sink.RingBuffer
        if isinstance(proc, io.sink.RingBuffer):
            return io.sink.PyReceiveModelFloat
        else:
            return super().select(proc, proc_models)

In [17]:
error = 0.0
size = 20
for i in range(size):
    target = train_data[i][1]
    run_condition = RunSteps(num_steps=nb_steps)
    run_config = Loihi1SimCfg(select_tag='fixed_pt', select_sub_proc_model=True)
    net.run(condition=run_condition, run_cfg=run_config) 
    results = output_logger.data.get()
    print(results)
    results = torch.Tensor(results).to(device)
    results = torch.unsqueeze(results, dim=0)
    print(type(results), results.shape)
    results = net_dnn(results)
    
    data = dataloader.data.get()
    print("input: ", data[:, 10])
    error += torch.sqrt((results.detach() - target) ** 2)
    print(results)
#gts = gt_logger.data.get()
print(error / size)
net.stop()

[[0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1.
  0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0.
  1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1.
  0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0.
  1. 1. 0. 1.]
 [0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
  0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1.
  1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1.
  1. 0. 1. 1.]]
<class 'torch.Tensor'> torch.Size([1, 2, 100])
input:  [0. 0. 0. 0. 1. 1. 0. 0.]
tensor([[0.2950, 0.2806]], grad_fn=<AddmmBackward0>)
[[1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0

In [32]:
print(results)

tensor([[0.4029, 0.5474]], grad_fn=<AddmmBackward0>)
